## Tarefa 2: Treinar um modelo

Na tarefa anterior, usamos o algoritmo integrado do SageMaker XGBoost. Este modo de algoritmo integrado do XGBoost não incorpora seu próprio script de treinamento do XGBoost e é executado diretamente nos conjuntos de dados de entrada. 

Nesta tarefa, vamos usar o XGBoost como framework. Ao executar o XGBoost como um framework no SageMaker, você obtém maior flexibilidade e acesso a cenários mais avançados, como validação cruzada k-fold, porque é possível personalizar seus próprios scripts de treinamento. 

Além de executar o trabalho de treinamento com um script correspondente, também vamos otimizar o treinamento executando um trabalho de ajuste de hiperparâmetros. Um hiperparâmetro é um parâmetro de alto nível que influencia o processo de aprendizagem durante o treinamento do modelo. Para obter as melhores previsões do modelo, você pode otimizar uma configuração de hiperparâmetros ou definir intervalos de hiperparâmetros. O processo de encontrar uma configuração ideal é chamado de ajuste de hiperparâmetros. 


### Tarefa 2.1: Configurar o ambiente

Antes de iniciar o treinamento do modelo, instale todas as dependências necessárias.


In [ ]:
import pandas as pd
import boto3
import sagemaker
import json
import joblib
from sagemaker.xgboost.estimator import XGBoost
from sagemaker.tuner import (
    IntegerParameter,
    ContinuousParameter,
    HyperparameterTuner
)
from sagemaker.inputs import TrainingInput
from sagemaker.image_uris import retrieve
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import CSVDeserializer

# Setting SageMaker variables
sess = sagemaker.Session()
region = sess.boto_region_name
s3_client = boto3.client("s3", region_name=region)
sagemaker_role = sagemaker.get_execution_role()
sagemaker_client = boto3.client("sagemaker")

 Em seguida, como antes, nós configuramos os caminhos de treinamento e validação que seu trabalho de treinamento usa como entrada.

In [ ]:
# define the bucket and prefix
#read_bucket = "<LAB_BUCKET>"
s3 = boto3.resource('s3')
for buckets in s3.buckets.all():
    if 'labdatabucket' in buckets.name:
        read_bucket = buckets.name
#read_bucket = "labdatabucket-us-west-2-110972467"
read_prefix = "scripts/data" 

# Setting S3 location for read and write operations
train_data_key = f"{read_prefix}/train/adult_data_processed_train_wheaders.csv"
test_data_key = f"{read_prefix}/test/adult_data_processed_test_wheaders.csv"
validation_data_key = f"{read_prefix}/validation/adult_data_processed_validation_wheaders.csv"

write_bucket = read_bucket
write_prefix = "script-mode/data"

model_key = f"{write_prefix}/model"
output_key = f"{write_prefix}/output"

train_data_uri = f"s3://{read_bucket}/{train_data_key}"
validation_data_uri = f"s3://{read_bucket}/{validation_data_key}"
test_data_uri = f"s3://{read_bucket}/{test_data_key}"
model_uri = f"s3://{write_bucket}/{model_key}"
output_uri = f"s3://{write_bucket}/{output_key}"
estimator_output_uri = f"s3://{write_bucket}/{write_prefix}/training_jobs"
bias_report_output_uri = f"s3://{write_bucket}/{write_prefix}/clarify-output/bias"
explainability_report_output_uri = f"s3://{write_bucket}/{write_prefix}/clarify-output/explainability"

### Tarefa 2.2: Configurar um objeto estimador

Configuração do trabalho de ajuste 

In [ ]:
training_job_name_prefix = "xgbtrain"
tuning_job_name_prefix = "xgbtune" 
xgb_model_name = "xgb-script-mode-model"
train_instance_count = 1
train_instance_type = "ml.m5.xlarge"

Configure hiperparâmetros estáticos, intervalos de hiperparâmetros e objeto estimador. 

Observe que, diferentemente da tarefa 1, nesta tarefa em modo de script, definimos um script personalizado para o trabalho de treinamento. Observe que usamos um script Python personalizado como ponto de entrada ao definir seu estimador do SageMaker abaixo. Espere um pouco e abra o arquivo xgboost_train.py personalizado.

No arquivo xgboost_train.py personalizado, vai ser exibida uma seção com a execução de uma técnica de validação cruzada. Isso não é possível com o algoritmo de treinamento pronto para uso do SageMaker XGBoost.

Além disso, observe que os intervalos de hiperparâmetros que definimos na próxima célula são os mesmos hiperparâmetros estáticos que usamos no treinamento integrado do XGBoost na primeira tarefa, mas, desta vez, definimos intervalos para o sintonizador testar valores diferentes desses hiperparâmetros para encontrar a combinação com a melhor métrica de objetivo final. 


In [ ]:
# Set static hyperparameters that will not be tuned
static_hyperparams = {  
                        "eval_metric" : "auc",
                        "objective": "binary:logistic",
                        "num_round": "5"
                      }

# hyperparameter ranges that will be tuned 
hyperparameter_ranges = {
    "max_depth": IntegerParameter(6, 9),
    "eta": ContinuousParameter(0.01, 0.03),
    "gamma": ContinuousParameter(0.5, 0.9),
    "min_child_weight": ContinuousParameter(0.5, 0.9),
    "subsample": ContinuousParameter(0.2, 0.5)
}

# XGBoost Estimator 
xgb_estimator = XGBoost(
                        entry_point="xgboost_train.py",
                        output_path=estimator_output_uri,
                        code_location=estimator_output_uri,
                        hyperparameters=static_hyperparams,
                        role=sagemaker_role,
                        instance_count=train_instance_count,
                        instance_type=train_instance_type,
                        framework_version="1.7-1",
                        base_job_name=training_job_name_prefix
                    )

Agora, crie um objeto de sintonizador que vai usar o estimador XGBoost, bem como os intervalos de hiperparâmetros que definimos. 

In [ ]:
objective_metric_name = "validation:auc"

# Setting up tuner object
tuner_config_dict = {
                     "estimator" : xgb_estimator,
                     "max_jobs" : 6,
                     "max_parallel_jobs" : 3,    
                     "objective_metric_name" : objective_metric_name,
                     "hyperparameter_ranges" : hyperparameter_ranges,
                     "base_tuning_job_name" : tuning_job_name_prefix,
                     "strategy" : "Random"
                    }
tuner = HyperparameterTuner(**tuner_config_dict)

Configurar os canais de entrada para o trabalho de ajuste e executar o trabalho do sintonizador 

In [ ]:
s3_input_train = TrainingInput(s3_data="s3://{}/{}".format(read_bucket, train_data_key), content_type="csv", s3_data_type="S3Prefix")
s3_input_validation = (TrainingInput(s3_data="s3://{}/{}".format(read_bucket, validation_data_key), content_type="csv", s3_data_type="S3Prefix"))

tuner.fit(inputs={"train": s3_input_train, "validation": s3_input_validation}, include_cls_metadata=False)
tuner.wait()

<i aria-hidden="true" class="fas fa-clipboard-check" style="color:#18ab4b"></i> **Saída esperada:** se a configuração do estimador e do hiperparâmetro estiver correta e o trabalho de ajuste do hiperparâmetro for concluído corretamente, vai ser exibido o seguinte resultado:

```plain
************************
**** EXEMPLO DE SAÍDA ****
************************

Não foi encontrado nenhum trabalho de treinamento concluído associado a este estimador. Verifique se esse estimador é usado apenas para criar a configuração do fluxo de trabalho
...................................!
!
```

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#ff6633"></i> **Observação:** o treinamento leva de três a quatro minutos para ser executado.


Execute um resumo dos resultados de ajuste ordenados em ordem decrescente de desempenho, onde o maior valor objetivo está na parte superior.  

In [ ]:
df_tuner = sagemaker.HyperparameterTuningJobAnalytics(tuner.latest_tuning_job.job_name).dataframe()
df_tuner = df_tuner[df_tuner["FinalObjectiveValue"]>-float('inf')].sort_values("FinalObjectiveValue", ascending=False)
df_tuner

### Limpeza

Você concluiu este caderno. Passe para a próxima parte do laboratório da seguinte forma:

– Feche este arquivo do caderno.
– Retorne à sessão de laboratório e conclua-o.